<a href="https://colab.research.google.com/github/JUSTKXIZEN/BASIC-IMAGE-AND-AUDIO-ENHANCER-USING-SKIT-LEARNING/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# Install required packages
!pip install scikit-image librosa soundfile scipy matplotlib noisereduce

# Corrected imports
import numpy as np
from skimage import io, img_as_float, img_as_ubyte
from skimage.restoration import denoise_nl_means
from skimage.exposure import equalize_adapthist
from skimage.filters import gaussian, unsharp_mask
import librosa
import soundfile as sf
from scipy.signal import butter, filtfilt
import noisereduce as nr
from google.colab import files
import matplotlib.pyplot as plt

In [16]:
class MediaEnhancer:
    # Adaptive Image Enhancer
    def enhance_image(image_path):
    # Read the image
        img = io.imread(image_path)

    # Convert to float for processing
        img_float = img_as_float(img)

    # Apply Non-local Means denoising with milder parameters
        denoised = denoise_nl_means(img_float, patch_size=5, patch_distance=7, h=0.05, multichannel=True)

    # Apply Contrast Limited Adaptive Histogram Equalization (CLAHE) with lower clip_limit
        contrast_enhanced = equalize_adapthist(denoised, clip_limit=0.01)

    # Apply unsharp mask for mild sharpening
        sharpened = unsharp_mask(contrast_enhanced, radius=1, amount=0.5)

    # Ensure pixel values are in valid range
        enhanced_img = np.clip(sharpened, 0, 1)

        return img_as_ubyte(enhanced_img)

    # High-pass filter for audio processing
    def butter_highpass(self, cutoff, fs, order=5):
        nyq = 0.5 * fs
        normal_cutoff = cutoff / nyq
        b, a = butter(order, normal_cutoff, btype='high', analog=False)
        return b, a

    # Adaptive Audio Enhancer
    def enhance_audio(self, audio_path, output_path):
        # Load the audio
        y, sr = librosa.load(audio_path, sr=44100)

        # High-pass filter for low-frequency noise
        b, a = self.butter_highpass(cutoff=100, fs=sr, order=4)
        y_highpassed = filtfilt(b, a, y)

        # Reduce noise using spectral gating
        y_denoised = nr.reduce_noise(y=y_highpassed, sr=sr, prop_decrease=0.8, stationary=True)

        # Dereverberation: Spectral subtraction
        spectrogram = librosa.stft(y_denoised)
        magnitude, phase = librosa.magphase(spectrogram)
        magnitude_cleaned = np.maximum(magnitude - 0.05 * np.median(magnitude), 0)
        y_dereverbed = librosa.istft(magnitude_cleaned * phase)

        # Normalize and enhance the audio
        y_normalized = librosa.util.normalize(y_dereverbed)

        # Save the enhanced audio
        sf.write(output_path, y_normalized, sr, 'PCM_24')


In [21]:
# Main function
def main():
    enhancer = MediaEnhancer()

    # Step 1: Process the Image
    print("Please upload your image file:")
    from google.colab import files
    uploaded_image = files.upload()  # Upload the image file

    image_file = next((name for name in uploaded_image.keys() if name.lower().endswith(('.jpg', '.png', '.jpeg'))), None)
    if image_file:
        try:
            original_image = io.imread(image_file)
            enhanced_image = enhancer.enhance_image(image_file)

            # Save and display enhanced image
            enhanced_image_path = "enhanced_image.jpg"
            io.imsave(enhanced_image_path, enhanced_image)

            print(f"Enhanced image saved to: {enhanced_image_path}")
            plt.figure(figsize=(10, 5))
            plt.subplot(1, 2, 1)
            plt.imshow(original_image)
            plt.title('Original Image')
            plt.axis('off')

            plt.subplot(1, 2, 2)
            plt.imshow(enhanced_image)
            plt.title('Enhanced Image')
            plt.axis('off')
            plt.show()
        except Exception as e:
            print(f"Error processing image: {e}")
    else:
        print("Error: No valid image file uploaded. Please upload a file with extensions .jpg, .png, or .jpeg.")

    # Step 2: Process the Audio
    print("Please upload your audio file:")
    uploaded_audio = files.upload()  # Upload the audio file

    audio_file = next((name for name in uploaded_audio.keys() if name.lower().endswith(('.wav', '.mp3'))), None)
    if audio_file:
        try:
            enhanced_audio_path = "enhanced_audio.wav"
            enhancer.enhance_audio(audio_file, enhanced_audio_path)
            print(f"Enhanced audio saved to: {enhanced_audio_path}")
        except Exception as e:
            print(f"Error processing audio: {e}")
    else:
        print("Error: No valid audio file uploaded. Please upload a file with extensions .wav or .mp3.")

    print("Enhancement completed!")

if __name__ == "__main__":
    main()

Please upload your image file:


Saving download.jpeg to download (6).jpeg
Error processing image: MediaEnhancer.enhance_image() takes 1 positional argument but 2 were given
Please upload your audio file:


Error: No valid audio file uploaded. Please upload a file with extensions .wav or .mp3.
Enhancement completed!
